# PR1: Medical Text Classification

* Author: Kevin Chuang [@k-chuang](https://www.github.com/k-chuang)
* Created on: September 21, 2018
* Description: Given a medical abstract, classify condition of patient (5 classes) using K-Nearest Neighbors.

-----------

## Import libraries

In [1]:
__author__ = 'Kevin Chuang (https://www.github.com/k-chuang)' 

# linear algebra
import numpy as np 

# data processing
import pandas as pd 

# data visualization
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

# Text Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

# Feature Selection
from sklearn.feature_selection import SelectKBest, chi2

# Natural Language Processing
from nltk import word_tokenize, WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer

# Metrics
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split

# Utilities
import string
import math
from operator import itemgetter 
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
import scipy.sparse as sp

%matplotlib inline


sns.set(rc={"figure.figsize": (20.0, 10.0), "axes.labelsize": 14})

# sns.set_context("poster")

## Load data

In [2]:
train_df = pd.read_csv('../train.dat', sep='\t', header=None, names=['Label', 'Abstract'])
test_df = pd.read_csv('../test.dat', sep='\t', header=None, names=['Abstract'])
submission_df = pd.read_csv('../format.dat', header=None, names=['Labels'])

In [3]:
train_df.head()

,Label,Abstract
0,4,Catheterization laboratory events and hospital...
1,5,Renal abscess in children. Three cases of rena...
2,2,Hyperplastic polyps seen at sigmoidoscopy are ...
3,5,Subclavian artery to innominate vein fistula a...
4,4,Effect of local inhibition of gamma-aminobutyr...


In [4]:
test_df.head()

,Abstract
0,Excision of limbal dermoids. We reviewed the c...
1,Bell's palsy. A diagnosis of exclusion. In cas...
2,Retained endobronchial foreign body removal fa...
3,Recurrent buccal space abscesses: a complicati...
4,Intracranial fibromatosis. Fibromatoses are un...


In [5]:
# Combine train and test abstracts to use for building vectorizer
abstract_df = pd.concat([train_df['Abstract'], test_df['Abstract']])

In [6]:
abstract_df.shape

(28880,)

## Exploratory Data Analysis

- See other notebook for detailed EDA 
    - [exploratory-data-analysis.ipynb]()

## Data Preprocessing

- Going to use `Bag of Words` approach
    - presence of words (frequency or count) is taken into consideration & order is ignored
    - BOW basically breaks up the note into the individual words and counts how many times each word occurs.
- Tokenizer and a vectorizer. 
    - The tokenizer breaks a single abstract into a list of words and a vectorizer takes a list of words and counts the words.
- `Tokenizer`:
    - Remove punctuation & numbers
    - Lowercase everything
    - Two approaches for tokenization
        - Goal is reduce inflectional forms and sometimes derivationally related forms of a word to a common base form.
        - `WordNetLemmatizer`
            - Lemmatize all the text (e.g. women will become woman)
                - Lemmatization is the process of converting the words of a sentence to its dictionary form. 
        - `PorterStemmer`
            - Stem all the text
                - Stemming is the process of converting words to the stem (root) of the word
- `Vectorizer`:
    - General process of turning a collection of text documents into numerical feature vectors. 
    - This specific strategy (tokenization, counting and normalization) is called the Bag of Words or “Bag of n-grams” representation. 
    - Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.
    - `CountVectorizer`
        - Encodes a vector with a length of the entire vocabulary and an integer count for the number of times each word appeared in the document.
        - `vocabulary_` is a dict/mapping of the terms to their indices in the document-term matrix, not the counts.
    - `TfidfVectorizer`
        - Convert a collection of raw documents to a matrix of TF-IDF features.
            - Normalizing and weighting with diminishing importance tokens that occur in the majority of samples / documents.
        - TF-IDF are word frequency scores that try to highlight words that are more interesting, e.g. frequent in a document but not across documents.
            - *This can be extremely useful for this problem, since we have 5 categories we are trying to classify and thus certain categories (medical conditions) may have words in the medical abstract that are unique to the condition*
        - The resulting tf-idf vectors are then normalized by the Euclidean norm (L2)
        - **Since the medical text data have a lot of multi-word expressions (e.g. *left anterior descending coronary artery*), I will use N-grams (where N >= 1) to keep the local positioning of these important words **
            - Experimented with N-grams, and it seems 1-gram, 2-gram, 3-gram, and 4-grams produce best results

In [7]:
def lemma_tokenizer(text):
    '''Tokenize text into a list of preprocessed words '''
    
    # Create a string with all punctuations & digits concatenated
    num_and_punc = string.punctuation + string.digits
    
    # Create a mapping to space using string above for each num/punc & return a translation table with mapping
    t_table = str.maketrans(dict.fromkeys(num_and_punc, " "))
    
    # Lower text and use translation table to remove all punctuation and digits
    text = text.lower().translate(t_table)
    
    # Use Lemma tokenizer to tokenize the words
    lemma = WordNetLemmatizer()
    lemmas = [lemma.lemmatize(word.strip()) for word in text.split()]
    
    return lemmas

def word_tokenizer(text):
    '''Tokenize text into a list of preprocessed words '''
    
    # Create a string with all punctuations & digits concatenated
    num_and_punc = string.punctuation + string.digits
    
    # Create a mapping to space using string above for each num/punc & return a translation table with mapping
    t_table = str.maketrans(dict.fromkeys(num_and_punc, " "))
    
    # Lower text and use translation table to remove all punctuation and digits
    text = text.lower().translate(t_table)
    
    tokens = word_tokenize(text)
    return tokens

def tokenizer(text):
    '''Tokenize text into a list of preprocessed words '''
    
    # Create a string with all punctuations & digits concatenated
    num_and_punc = string.punctuation + string.digits
    
    # Create a mapping to space using string above for each num/punc & return a translation table with mapping
    t_table = str.maketrans(dict.fromkeys(num_and_punc, " "))
    
    # Lower text and use translation table to remove all punctuation and digits
    text = text.lower().translate(t_table)
    # Best Stemmer for this dataset (Tested)
    stemmer = PorterStemmer()
#     stemmer = SnowballStemmer("english")
#     stemmer = LancasterStemmer()
    stems = [stemmer.stem(word.strip()) for word in text.split()]
    return stems

In [8]:
# Let's create stop words list & remove unimportant words such as 'the' or 'and'

# 153 stop words from NLTK
nltk_stop_words = stopwords.words('english')
# Combine stop words from all the stop word lists
stop_words = ENGLISH_STOP_WORDS.union(nltk_stop_words)

In [9]:
# 381 stop words
len(stop_words)

378

In [10]:
# Using tf-idf

tfidf_vec = TfidfVectorizer(tokenizer = tokenizer, norm='l2', ngram_range=(1,2), sublinear_tf = True, min_df = 5,
                            stop_words = stop_words)
tfidf_vec.fit(abstract_df.values)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=frozenset({'made', 'even', 'sometime', 've', 'do', 'won', 'she', 'these', 'empty', 'others', 'every', 'anyone', 'became', 'nothing', 'already', 'rather', 'much', 'other', "hasn't", 'formerly', 'sometimes', 'amoungst', 'whole', 'at', 'keep', 'mill', 'are', 'everyone', 'both', 'around', 'fe...that'll", 'hereupon', 'sincere', 'seems', 'go', 'who', 'during', 'toward', 'himself', 'back', 'as'}),
        strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function tokenizer at 0x105dcaea0>, use_idf=True,
        vocabulary=None)

In [11]:
X_train_tfidf = tfidf_vec.transform(train_df['Abstract'].values)
X_test_tfidf = tfidf_vec.transform(test_df['Abstract'].values)

In [12]:
print('There are %i features' % X_train_tfidf.shape[1])

There are 139118 features


In [13]:
Y_train = train_df['Label'].values

## Implement k-NN classifier

- KNN is a lazy, nonparametric, instance based classifier 
- Using cosine distance as the distance metric for kNN
    - Good when using bag of words with tf-idf
    - Have to normalize vectors when using cosine similarity (which `TfidfVectorizer` does already with L2 normalization)
- Uses majority voting of K nearest neighbors to determine the label of an instance
- If there is a stalemate, I use the instances of the labels that tied, and calculate their inverse distance squared to determine the final winner
    - A higher inverse squared distance score correlates with a higher similarity (farther distance squared)

In [14]:
"""
Distance Metrics.

Compute the distance between two items (usually strings).
As metrics, they must satisfy the following three requirements:

1. d(a, a) = 0
2. d(a, b) >= 0
3. d(a, c) <= d(a, b) + d(b, c)
"""
    
def cosine_similarity_sparse(s1, s2):
    '''Calculate cosine similiarity of two sparse matrices'''
    # Calculate dot product (already L2 norm vectors so we do not need to divide)
    numerator = s1.dot(s2.T)
    return numerator

def cosine_distance_sparse(s1, s2):
    '''Calculate cosine distance of two sparse matrices (1 - cosine_similarity)'''
    # Calculate dot product (already L2 norm vectors so we do not need to divide)
    cos_sim = s1.dot(s2.T)
    if s1.shape[0] > s2.shape[0]:
        one_array = np.ones((s1.shape[0], 1), dtype=float)
    else:
        one_array = np.ones((s2.shape[0], 1), dtype=float)
    return csr_matrix(one_array - cos_sim)

In [15]:
def split_data(features, labels, fold_num = 1, fold=10):
    n = features.shape[0]
    fold_size = int(np.ceil(n*1.0/fold))
    feats = []
    cls_train = []
    for f in range(fold):
        if f+1 != fold_num:
            feats.append(features[f*fold_size: min((f+1)*fold_size, n)])
            cls_train.extend(labels[f*fold_size: min((f+1)*fold_size, n)])
    # join all fold matrices that are not the test matrix
    train = sp.vstack(feats, format='csr')
    # extract the test matrix and class values associated with the test rows
    test = features[(fold_num-1)*fold_size: min(fold_num*fold_size, n), :]
    cls_test = labels[(fold_num-1)*fold_size: min(fold_num*fold_size, n)]
    return train, cls_train, test, cls_test

In [16]:
# Metrics (Accuracy & F1 Score Functions)

def calculate_accuracy(label, prediction):
    '''Takes two numpy arrays or Python lists and produces an accuracy score in %'''
    if isinstance(label, np.ndarray) and isinstance(prediction, np.ndarray):
        assert label.shape == prediction.shape
        return (label == prediction).all().mean() * 100.0
    elif isinstance(label, list) and isinstance(prediction, list):
        assert len(label) == len(prediction)
        return sum(1 for a,b in zip(label, prediction) if a == b) / len(label)
    else:
        raise AttributeError('Both arguments have to be lists or numpy arrays')

def calculate_weighted_f1_score(label, prediction):
    if isinstance(label, np.ndarray) or isinstance(prediction, np.ndarray):
        label = label.tolist()
        prediction = prediction.tolist()
    f1_list = []
    label_dict = Counter(label)
    label_dict = sorted(label_dict.items(), key=lambda x: x[0])
    for l, support in label_dict:
        tp = 0.
        fp = 0.
        tn = 0.
        fn = 0.
        for i in range(len(label)):
            if prediction[i] == l:
                if prediction[i] == label[i]:
                    tp += 1.
                else:
                    fp += 1.
            else:
                if label[i] == l:
                    fn += 1.
        # precision is "how useful the search results are"
        precision = tp / (tp + fp)
        # recall is "how complete the results are"
        recall = tp / (tp + fn)
        
        if precision == 0.0 or recall == 0.0:
            f1_score = 0.0
        else:
            f1_score = 2*((precision*recall)/(precision+recall))
        weighted_f1_score = f1_score * support
        f1_list.append(weighted_f1_score)
    return sum(f1_list) / len(label)

In [17]:
def classify_condition(train, labels, instance, K=5, metric = 'cosine'):
    '''Using a distance metric to classify an instance'''
    if metric == 'cosine':
        dots = cosine_distance_sparse(train, instance)
    elif metric == 'euclidean':
        dots = euclidean_distance_sparse(train, instance)
    # Edit below later on
    elif metric == 'jaccard':
        dots = csr_matrix(cdist(train.toarray(), instance.toarray(), 'jaccard'))
    else:
        dots = cosine_distance_sparse(train, instance)
        
        
    neighbors = list(zip(dots.indptr, dots.data))
    if len(neighbors) == 0:
        # could not find any neighbors
        print('Could not find any neighbors.... Choosing a random one')
        return np.asscalar(np.random.randint(low=1, high=5, size=1))
    neighbors.sort(key=lambda x: x[1], reverse=False)        
        
    tc = Counter(labels[s[0]] for s in neighbors[:K]).most_common(5)
    
    if len(tc) == 1 or tc[0][1] > tc[1][1]:
        # majority vote
        return tc[0][0]

    # tie break
    # Only keep tied labels
    num_n = tc[0][1]
    keep_labels = [l for l, c in tc if c == num_n]
    tc = defaultdict(float)
     # Distance-Weighted Voting 
    for s in neighbors[:K]:
        if labels[s[0]] not in keep_labels:
            continue
        else:
            tc[labels[s[0]]] += (1 / (s[1]**2))
            
    return sorted(tc.items(), key=lambda x: x[1], reverse=True)[0][0]

In [18]:
def evaluate_model(features, labels, metric='cosine', K=3, fold=10):
    '''Using KFold Cross Validation to evaluate model accuracy'''
    
    if metric not in ['cosine', 'euclidean', 'jaccard', 'hamming', 'mahalanobis']:
        raise ValueError('Metric must be `cosine`, `euclidean`, or `jaccard`')
    
    macc = 0.0
    cum_f1 = 0.0
    for f in range(fold):
        # split data into training and testing
        train_set, train_labels, test_set, test_labels = split_data(features, labels, f+1, fold)
        # predict the class of each test sample
        predictions = np.array([classify_condition(train_set, train_labels, test_set[i,:], K=K, metric=metric) 
                       for i in range(test_set.shape[0])])
        acc = calculate_accuracy(test_labels, predictions)
#         f1 = calculate_weighted_f1_score(test_labels, predictions)
        f1 = f1_score(test_labels, predictions, average='weighted')
#         print('Fold-%i Accuracy: %.05f' % (f+1, acc))
        print('Fold-%i F1 Score: %.05f' % (f+1, f1))
        macc += acc
        cum_f1 += f1
    
    return macc/float(fold), cum_f1/float(fold)   

### F1 Score 10-Fold CV Total Time Taken

- My F1 score implementation (with Python lists): 17 minutes 5 seconds
- Sklearn's F1 score implementation (with sparse matrices): 16 minutes 50 seconds

## Tune hyperparameter K

- Implementation of simple grid search through various values of K

In [28]:
def grid_search(features, labels, start, end, inc=1):
    '''My Grid Search Function'''
    best_f1 = 0.0
    best_k = 0
    best_acc = 0.0
    for k in np.arange(start, end, inc):
        acc, f1 = evaluate_model(features, labels, K=k, fold=10)
#         print('For %i-NN, 10-Fold CV Average Accuracy: %.05f%%' % (k, acc * 100.0)) 
        print('For %i-NN, 10-Fold CV Weighted F1 Score: %.05f' % (k, f1)) 
        if f1 > best_f1:
            best_f1 = f1
            best_acc = acc
            best_k = k
    
    print('Best Model Params: \n For %d-NN, 10-Fold CV Weighted F1 Score: %.08f'% (best_k, best_f1))
    return best_k, best_acc, best_f1

In [29]:
K, acc, f1 = grid_search(X_train_tfidf, Y_train, start=10, end=80, inc=10)

Fold-1 F1 Score: 0.58120


KeyboardInterrupt: 

In [ ]:
K, acc, f1 = grid_search(X_train_tfidf, Y_train, start=30, end=60, inc=1)

## Prediction & Submission

In [ ]:
def predict_condition(X_train, Y_train, X_test, K = 69):
    predictions = np.array([classify_condition(X_train, Y_train, X_test[i,:], K=K, metric='cosine') 
                       for i in range(X_test.shape[0])])
    return predictions

In [ ]:
final_predictions = predict_condition(X_train_tfidf, Y_train, X_test_tfidf, K = K)

In [ ]:
submission_df['Labels'] = final_predictions

In [ ]:
submission_df.to_csv('submission.txt', sep='\n', index=False, header=False)